In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import string, os 
import tensorflow as tf

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Loading data as data frame.
df = pd.read_csv('drive/MyDrive/lyrics-data.csv')
df.head()

,ALink,SName,SLink,Lyric,language
0,/ivete-sangalo/,Arerê,/ivete-sangalo/arere.html,"Tudo o que eu quero nessa vida,\nToda vida, é\...",pt
1,/ivete-sangalo/,Se Eu Não Te Amasse Tanto Assim,/ivete-sangalo/se-eu-nao-te-amasse-tanto-assim...,Meu coração\nSem direção\nVoando só por voar\n...,pt
2,/ivete-sangalo/,Céu da Boca,/ivete-sangalo/chupa-toda.html,É de babaixá!\nÉ de balacubaca!\nÉ de babaixá!...,pt
3,/ivete-sangalo/,Quando A Chuva Passar,/ivete-sangalo/quando-a-chuva-passar.html,Quando a chuva passar\n\nPra quê falar\nSe voc...,pt
4,/ivete-sangalo/,Sorte Grande,/ivete-sangalo/sorte-grande.html,A minha sorte grande foi você cair do céu\nMin...,pt


In [5]:
# Dropping unnecessary columns.
df.drop(['ALink','SName','SLink'],axis=1,inplace=True)

In [6]:
# value count
df['language'].value_counts()

en     191814
pt     157393
es       9917
rw       1679
it       1432
fr       1225
de        844
fi        145
sv        112
ro         97
no         89
is         86
tl         69
pl         47
gl         36
ga         32
tr         32
id         26
cy         23
su         19
af         19
sw         19
ko         17
nl         14
da         13
ca         13
et         13
ms          8
ja          7
st          7
ht          5
gd          4
ru          4
ar          4
eu          4
cs          3
ku          3
ny          3
mg          3
lg          2
jw          2
hu          2
vi          1
iw          1
sr          1
hmn         1
hr          1
fa          1
sq          1
zh          1
lv          1
sl          1
Name: language, dtype: int64

In [7]:
#Exttracting just English language songs as we will be generating English lyrics only.
df = df[df['language']=='en']

In [8]:
df.columns

Index(['Lyric', 'language'], dtype='object')

In [ ]:
pip install textblob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 160, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 241, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 499, in run
    confl

In [9]:
# Calculating a sentiment score with 0 being the threshold. 
from textblob import TextBlob

def sentiment_label(text):
    sentiment_score = TextBlob(text).sentiment.polarity
    if sentiment_score > 0:
        return 'happy'
    elif sentiment_score < 0:
        return 'sad'
    else:
        return 'neutral'

df['sentiment'] = df['Lyric'].apply(sentiment_label)

<ipython-input-9-40d5d5f38076>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment'] = df['Lyric'].apply(sentiment_label)


In [10]:
df.head()

,Lyric,language,sentiment
69,I feel so unsure\nAs I take your hand and lead...,en,happy
86,"Don't let them fool, ya\nOr even try to school...",en,happy
88,"Baby, let's cruise, away from here\nDon't be c...",en,happy
111,"Know it sounds funny\nBut, I just can't stand ...",en,happy
140,You've got that look again\nThe one I hoped I ...,en,happy


In [11]:
df['sentiment'].value_counts()

happy      134386
sad         54946
neutral      2482
Name: sentiment, dtype: int64

In [12]:
#In the next four steps we are doing downsampling as the number of "happy" instances are very high in comparison to the "sad" andd "neutral" ones. 
rows_to_remove = df[df['sentiment'] == 'happy'].sample(n=79000)

In [13]:
rows_to_remove['sentiment'].value_counts()

happy    79000
Name: sentiment, dtype: int64

In [14]:
df_modified = df.drop(rows_to_remove.index)

In [15]:
df_modified['sentiment'].value_counts()

happy      55386
sad        54946
neutral     2482
Name: sentiment, dtype: int64

In [16]:
#In the next four steps we reomve all the "neutral" instances as they are too low
neutral_senti_rows_to_remove = df[df['sentiment'] == 'neutral']

In [17]:
neutral_senti_rows_to_remove['sentiment'].value_counts()

neutral    2482
Name: sentiment, dtype: int64

In [18]:
df_modified = df_modified.drop(neutral_senti_rows_to_remove.index)

In [19]:
df_modified['sentiment'].value_counts()

happy    55386
sad      54946
Name: sentiment, dtype: int64

In [20]:
df_modified.head()

,Lyric,language,sentiment
86,"Don't let them fool, ya\nOr even try to school...",en,happy
88,"Baby, let's cruise, away from here\nDon't be c...",en,happy
111,"Know it sounds funny\nBut, I just can't stand ...",en,happy
140,You've got that look again\nThe one I hoped I ...,en,happy
168,Everyone's feeling pretty\nIt's hotter than Ju...,en,happy


In [22]:
#We are now modifying our data set as such --> "<happy/sad>: <yrics>"
df_modified['lyrics_with_sentiment'] = df_modified['sentiment'] + ": " + df_modified['Lyric']

In [23]:
df_modified.head()

,Lyric,language,sentiment,lyrics_with_sentiment
86,"Don't let them fool, ya\nOr even try to school...",en,happy,"happy: Don't let them fool, ya\nOr even try to..."
88,"Baby, let's cruise, away from here\nDon't be c...",en,happy,"happy: Baby, let's cruise, away from here\nDon..."
111,"Know it sounds funny\nBut, I just can't stand ...",en,happy,"happy: Know it sounds funny\nBut, I just can't..."
140,You've got that look again\nThe one I hoped I ...,en,happy,happy: You've got that look again\nThe one I h...
168,Everyone's feeling pretty\nIt's hotter than Ju...,en,happy,happy: Everyone's feeling pretty\nIt's hotter ...


In [24]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.4 MB/s eta 0:00:00


In [25]:
#Training set (80/20 split)
df_modified_truncated = df_modified.iloc[:2000]

#Vaidation set
df_modified_validation_set = df_modified[2000:2500]

In [26]:
df_modified_truncated['lyrics_with_sentiment'].to_csv('lyrics_dataset_modified.txt', index=False, header=None)

In [27]:
df_modified_validation_set['lyrics_with_sentiment'].to_csv('lyrics_validation_set.txt', index=False, header=None)

In [29]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling
from torch.utils.data import DataLoader

# Load pre-trained GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Tokenize dataset and create TextDataset
def load_dataset(file_path, tokenizer):
    dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=128
    )
    return dataset

train_dataset = load_dataset('lyrics_dataset_modified.txt', tokenizer)
validation_dataset = load_dataset('lyrics_validation_set.txt', tokenizer)

# Data collator to create batches
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)

In [35]:
from transformers import Trainer, TrainingArguments

# Load pre-trained GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Define training arguments
training_args = TrainingArguments(
    output_dir="./output",
    overwrite_output_dir=True,
    num_train_epochs=10,
    evaluation_strategy="epoch",
    per_device_train_batch_size=4,
    save_steps=100,
    save_total_limit=2,
    logging_dir='./logs',
)

# Create a Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
)

# Fine-tune the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,3.179900,3.640633
2,2.961100,3.642054
3,2.789800,3.666516
4,2.639000,3.699453
5,2.541300,3.740723
6,2.419600,3.777518
7,2.336100,3.825857
8,2.292300,3.875850
9,2.261000,3.888371
10,2.216300,3.907690


TrainOutput(global_step=25850, training_loss=2.570149166616296, metrics={'train_runtime': 2313.9615, 'train_samples_per_second': 44.685, 'train_steps_per_second': 11.171, 'total_flos': 6754399027200000.0, 'train_loss': 2.570149166616296, 'epoch': 10.0})

In [36]:
trainer.save_model("./output")

In [39]:
eval_results = trainer.evaluate()

# Calculate perplexity
import math
perplexity = math.exp(eval_results["eval_loss"])
print("Perplexity:", perplexity)

Perplexity: 49.783832694649114


In [37]:
from transformers import pipeline

# Load the fine-tuned model and create a text generation pipeline
generator = pipeline('text-generation', model='./output', tokenizer=tokenizer)

# Generate new lyrics with a sentiment prompt
generated_text = generator("sad: ", max_length=100, num_return_sequences=1)[0]['generated_text']
print(generated_text)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


sad: ive seen the best of both worlds
the worlds of cleveys and sandals
nascar, planes, boats
birds eye, laser rays
aint nothing like tennis shoes
I know, my eyes, I'm blessed with eyes
smellin like whiskey
all through my heart
cause all that matter is love

Chorus:
If i tell you right, god will know
if i'm right this time
if i'm wrong this time



In [38]:
# Generate new lyrics with a sentiment prompt
generated_text = generator("happy: ", max_length=100, num_return_sequences=1)[0]['generated_text']
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


happy: ive never been afraid
I will always remember you
My first love song
I will always remember you
Even though hate will blind us and tear us apart
I will Tigger every step of the way

No matter what, I will never walk away
As you always will
I will always remember you
My first love song
I will always remember you
Even though hate will blind us and tear us apart
I will Tigger every step of the way

